In [1]:
from langchain.schema import Document
from langchain.document_transformers import DoctranTextTranslator

In [2]:
!nvidia-smi

Mon Aug 21 11:05:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   43C    P0              29W /  70W |      2MiB / 15360MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.268
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
!ls -lah /root/.cache/huggingface/hub/

total 4.0K
drwxr-xr-x 4 root root 108 2023-08-21__09:16:52 .
drwxr-xr-x 3 root root  30 2023-08-16__14:49:17 ..
drwxr-xr-x 6 root root  65 2023-08-21__09:16:52 models--bigscience--bloom-1b7
drwxr-xr-x 6 root root  65 2023-08-16__17:33:10 models--meta-llama--Llama-2-7b-chat-hf
-rw-r--r-- 1 root root   1 2023-07-26__16:10:47 version.txt


In [5]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [6]:
import torch
torch.cuda.device_count()

1

In [7]:
torch.device("cuda")

device(type='cuda')

In [8]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [9]:
# https://huggingface.co/bigscience/bloom-1b7

In [87]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [75]:
from transformers import BloomTokenizerFast

In [103]:
# tokenizer = BloomTokenizerFast.from_pretrained(
#     "bigscience/bloom-1b7",
#     local_files_only=True
# )

In [104]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="bigscience/bloom-1b7",
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )

In [105]:
tokenizer.eos_token_id

2

In [106]:
tokenizer.pad_token_id

3

In [79]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_build_conversation_input_ids',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_toke

In [89]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b7",
                                             local_files_only=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             temperature=0
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

In [90]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Aug 21 11:38:07 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

In [91]:
# from langchain.llms import HuggingFacePipeline
# import torch
# llm = HuggingFacePipeline.from_model_id(
#     model_id="bigscience/bloom-1b7",
#     task="text-generation",
#     model_kwargs={"temperature": 0, "max_length": 1024},
#     # device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# )

In [12]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [13]:
# https://python.langchain.com/docs/modules/model_io/models/chat/prompts

In [107]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [108]:
template = """Use the following pieces of context to answer the question at the end. 
  If you don't know the answer, just say that you don't know, don't try to make up an answer.
  {context}

  Question: {question}
  Answer:"""

In [116]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    template
)

In [124]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [110]:
prompt = prompt_template.format(
        # output_language="Tranditional Chinese",
        # output_language="English",
        # output_language="Chinese",
        context='My name is Ethan. I live in Taiwan.',
        question="What is my name?"
    )

In [125]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.pad_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

In [126]:
def llama_generate(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs,
                            max_new_tokens=512,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            )
    final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # final_outputs = cut_off_text(final_outputs, '')
    final_outputs = remove_substring(final_outputs, prompt)
    return final_outputs

In [127]:
llama_generate(prompt)

' My name is Ethan. I live in Taiwan.\n\nA:\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n What is my name?\n\nThe answer is\n\n My name is Ethan. I live in Taiwan.\n\nThe question is\n\n Wha

In [123]:
llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 512},
)

Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


In [84]:
chain = prompt_template | llm

# 1

In [51]:
context1 = """
  2 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  
SOP  
I. This guide will walk you through the process of requesting  RMA  through the 
Wareconn Customer Portal.  
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)  
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal3 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure  
2. Go to  RMA Request  Page 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims  
⚫ Description: Go to RMA request page  
 
To go to the RMA request page, follow these steps:  
Click Warranty Claims  in the left menu, then click +Add  
 
Click the icon shown in the image below  
(Select the corresponding SI )4 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure  
3. Create RMA Request  
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request  
⚫ Description: Create RMA request order  
 
To create a RMA request, follow these steps:  
Click Add Warranty Claims   
 
Then, click Parts Return
"""

In [52]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [53]:
prompt1 = prompt_template.format(
        # output_language="Tranditional Chinese",
        # output_language="English",
        # output_language="English",
        context=context1,
        question=question1
    )

In [54]:
prompt1

"Use the following pieces of context to answer the question at the end. \n  If you don't know the answer, just say that you don't know, don't try to make up an answer.\n  \n  2 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  \nSOP  \nI. This guide will walk you through the process of requesting  RMA  through the \nWareconn Customer Portal.  \nII. Details on how to fill in the Part Number (PN) and Serial Number (SN)  \n1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareconn.com/  \n⚫ Description:  Login  with account and password  \n \nTo log in, follow these steps:  \nGo to https://www.wareconn.com/ and click Log in  \n \nFill in your email and password, then click Login\n \nClick Enter  in the Customer portal3 / 6 \nThis document  belongs to Wareconn  

In [85]:
%%time
llm(
    prompt=prompt1,
    max_new_tokens=512,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

CPU times: user 10min 51s, sys: 4.46 s, total: 10min 55s
Wall time: 54.8 s


' Click the RMA Request  button in the left menu, then click +Add  \n \nClick the icon shown in the image below  \n(Select the corresponding SI )5 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  5 wareconn standard operating procedure  \n4. Click the RMA Request  button in the left menu, then click +Add  \n \nClick the icon shown in the image below  \n(Select the corresponding SI )6 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  \n3. Click the RMA Request  button in the left menu, then click +Add  \n \nClick the icon shown in the image below  \n(Select the corresponding SI )7 / 6 \nThis document  belongs to Wareconn  Technology Services

In [86]:
%%time
chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : context1,
        "question": question1,
        "eos_token_id"=tokenizer.eos_token_id,
        "pad_token_id"=tokenizer.pad_token_id
    }
)

SyntaxError: ':' expected after dictionary key (<unknown>, line 8)

# 2

In [58]:
context2 = """
  6 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  
5. Submit the RMA  request  
 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit  
⚫ Description: Submit the RMA request  
 
The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process2 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  
SOP  
I. This guide will walk you through the process of requesting  RMA  through the 
Wareconn Customer Portal.  
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)  
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal4 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure  
3. Create RMA Request  
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request  
⚫ Description: Create RMA request order  
 
To create a RMA request, follow these steps:  
Click Add Warranty Claims   
 
Then, click Parts Return
"""

In [59]:
question2 = "What does the Reminder field indicate when submitting an RMA request?"

In [62]:
prompt2 = prompt_template.format(
        # output_language="Tranditional Chinese",
        # output_language="English",
        # output_language="English",
        context=context2,
        question=question2
    )
print(prompt2)

Use the following pieces of context to answer the question at the end. 
  If you don't know the answer, just say that you don't know, don't try to make up an answer.
  
  6 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  
5. Submit the RMA  request  
 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit  
⚫ Description: Submit the RMA request  
 
The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process2 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not us

In [73]:
%%time
llm(
    prompt=prompt2, 
    # max_new_tokens=512
)

CPU times: user 10min 38s, sys: 4.06 s, total: 10min 42s
Wall time: 53.7 s


' The Reminder field will show you whether this RMA request is approvable  \n(acceptable)  for warranty provider  or not.  \nPlease remember to fill in WAL# in Customer No. field  \n \n  Question: What does the Warranty Claims field indicate when submitting an RMA request?\n  Answer: The Warranty Claims field will show you the warranty provider name and the warranty type.   \n \n  Question: What does the Serial Number field indicate when submitting an RMA request?\n  Answer: The Serial Number field will show you the serial number of the part that you are requesting.   \n \n  Question: What does the Part Number (PN) field indicate when submitting an RMA request?\n  Answer: The Part Number (PN) field will show you the part number of the part that you are requesting.   \n \n  Question: What does the Warranty Claims (WCC) field indicate when submitting an RMA request?\n  Answer: The Warranty Claims (WCC) field will show you the warranty provider name and the warranty type.   \n \n  Questio

In [74]:
%%time
chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : context2,
        "question": question2
    }
)

CPU times: user 10min 36s, sys: 4.22 s, total: 10min 40s
Wall time: 53.6 s


' The Reminder field will show you whether this RMA request is approvable  \n(acceptable)  for warranty provider  or not.  \nPlease remember to fill in WAL# in Customer No. field  \n \n  Question: What does the Warranty Claims field indicate when submitting an RMA request?\n  Answer: The Warranty Claims field will show you the warranty provider name and the warranty type.   \n \n  Question: What does the Serial Number field indicate when submitting an RMA request?\n  Answer: The Serial Number field will show you the serial number of the part that you are requesting.   \n \n  Question: What does the Part Number (PN) field indicate when submitting an RMA request?\n  Answer: The Part Number (PN) field will show you the part number of the part that you are requesting.   \n \n  Question: What does the Warranty Claims (WCC) field indicate when submitting an RMA request?\n  Answer: The Warranty Claims (WCC) field will show you the warranty provider name and the warranty type.   \n \n  Questio

In [22]:
prompt1="""
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  2 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  
SOP  
I. This guide will walk you through the process of requesting  RMA  through the 
Wareconn Customer Portal.  
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)  
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal3 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure  
2. Go to  RMA Request  Page 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims  
⚫ Description: Go to RMA request page  
 
To go to the RMA request page, follow these steps:  
Click Warranty Claims  in the left menu, then click +Add  
 
Click the icon shown in the image below  
(Select the corresponding SI )4 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure  
3. Create RMA Request  
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request  
⚫ Description: Create RMA request order  
 
To create a RMA request, follow these steps:  
Click Add Warranty Claims   
 
Then, click Parts Return

  Question: How can I navigate to the RMA request page in the Wareconn Customer Portal?
  Answer:
"""

In [23]:
llm.generate([prompt1])

/usr/local/lib/python3.10/site-packages/transformers/generation/utils.py:1262: UserWarning: Input length of input_ids is 515, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


LLMResult(generations=[[Generation(text='1', generation_info=None)]], llm_output=None, run=[RunInfo(run_id=UUID('943d2026-09bc-4946-9c74-a8e7811f86bc'))])

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "Use the following pieces of context to answer the question at the end."
                "If you don't know the answer, just say that you don't know, don't try to make up an answer."
                "Answer the question in {output_language} language."
                "--------------"
                "{context}"
            )
        ),
        HumanMessagePromptTemplate.from_template(
            "Question: {question}"
            # "Answer:"
        ),
    ]
)

template.format_messages(
    output_language="Tranditional Chinese",
    context='My name is Ethan. I live in Taiwan.',
    question="What is my name?"
)

[SystemMessage(content="Use the following pieces of context to answer the question at the end.If you don't know the answer, just say that you don't know, don't try to make up an answer.Answer the question in {output_language} language.--------------{context}", additional_kwargs={}),
 HumanMessage(content='Question: What is my name?', additional_kwargs={}, example=False)]